In [109]:
!pip install openai
!pip install requests
!pip install tenacity

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [110]:
import json
import openai
import requests
from tenacity import retry, wait_random_exponential, stop_after_attempt

GPT_MODEL = "gpt-3.5-turbo-0613"
openai.api_key = ''

In [111]:
@retry(wait=wait_random_exponential(min=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, functions=None, function_call=None, model="gpt-3.5-turbo-0613",temperature=0,max_tokens=None):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + openai.api_key,
    }
    json_data = {"model": model, "messages": messages, "temperature": temperature,"max_tokens": max_tokens}

    if functions is not None:
        json_data.update({"functions": functions})
    if function_call is not None:
        json_data.update({"function_call": function_call})
    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

In [112]:
biomed_decision_tree = {
    'questions': ['What is your primary symptom?'],
    'answers': {
        'fever': {
            'questions': ['Is your fever above 100.4F?'],
            'answers': {
                'yes': {
                    'questions': ['Do you have other symptoms like cough or difficulty breathing?'],
                    'answers': {
                        'yes': 'Possible COVID-19. Please get tested immediately.',
                        'no': 'Possible flu. Rest, hydrate, and consult with a healthcare professional if symptoms persist.'
                    }
                },
                'no': {
                    'questions': ['Do you have a rash?'],
                    'answers': {
                        'yes': 'Possibly a mild allergic reaction. Monitor symptoms and contact healthcare professional if condition worsens.',
                        'no': 'It might be a mild infection. Rest and hydrate, but consult with a healthcare professional if symptoms persist.'
                    }
                }
            }
        },
        'cough': {
            'questions': ['Is your cough dry or productive?'],
            'answers': {
                'dry': {
                    'questions': ['Do you have a fever or difficulty breathing?'],
                    'answers': {
                        'yes': 'Possible COVID-19. Please get tested immediately.',
                        'no': 'Possibly a cold or allergies. Rest and hydrate, and consult a healthcare professional if symptoms persist.'
                    }
                },
                'productive': {
                    'questions': ['Are you experiencing shortness of breath?'],
                    'answers': {
                        'yes': 'Possibly pneumonia. Please consult with a healthcare professional immediately.',
                        'no': 'Possibly bronchitis. Rest, hydrate, and consult a healthcare professional if symptoms persist.'
                    }
                }
            }
        },
        'headache': {
            'questions': ['Is the headache severe?', 'Do you have nausea or vomiting?'],
            'answers': {
                'yes': 'Possibly migraine or more serious conditions. Please consult with a healthcare professional immediately.',
                'no': 'Possibly tension headache. Rest, hydrate, and consult a healthcare professional if symptoms persist.'
            }
        }
    }
}


In [113]:
class BiomedQATree:
    def __init__(self, decision_tree):
        self.__tree = decision_tree
        self.__current_node = self.__tree
        self.__current_question_index = 0
        self.__current_question = self.__get_current_question()
        self.__possible_answers = self.__get_current_possible_answers()

    def __get_current_question(self):
        return self.__current_node['questions'][self.__current_question_index]

    def __get_current_possible_answers(self):
        return list(self.__current_node['answers'].keys())

    def get_follow_up_q(self, answer):
        if answer in self.__current_node['answers']:
            next_node = self.__current_node['answers'][answer]

            if isinstance(next_node, str):
                advice = next_node
                self.__current_node = self.__tree
                self.__current_question_index = 0
                self.__current_question = self.__get_current_question()
                self.__possible_answers = self.__get_current_possible_answers()
                return advice
            else:
                self.__current_node = next_node
                self.__current_question_index = 0
                self.__current_question = self.__get_current_question()
                self.__possible_answers = self.__get_current_possible_answers()
                return self.__current_question
        else:
            raise ValueError("Invalid answer, try again.")

        # Increment the current question index after returning the current question
        if self.__current_question_index < len(self.__current_node['questions']) - 1:
            self.__current_question_index += 1
        else:
            self.__current_question_index = 0

    def get_current_question(self):
        return self.__current_question

    def get_current_possible_answers(self):
        return self.__possible_answers


In [114]:
def generate_schema(biomed_qa_tree):
    current_question = biomed_qa_tree.get_current_question()
    current_possible_answers = biomed_qa_tree.get_current_possible_answers()

    schema_template = {
        "name": "get_follow_up_q",
        "description": f"Given the current question '{current_question}', we try that the user gives us the answer. If we can consider the current question responded and retrieve the follow up question or final advice. Do not call this function if the user hasn't responded to the current question yet. Ask again or different is necessary but remember the current question {current_question}",
        "call": f"biomed_qa_tree.get_follow_up_question({{answer: 'user answer'}})",
        "parameters": {
            "type": "object",
            "properties": {
                "answer": {
                    "type": "string",
                    "enum": current_possible_answers,
                    "description": "The answer to the current question."
                }
            },
            "required": ["answer"]
        }
    }

    return schema_template


In [115]:
def chat():
    tree= BiomedQATree(biomed_decision_tree)
    messages = [None, {"role": "assistant", "content": "Hello, how can I help you today?"}]
    while True:
        functions = [generate_schema(tree)]
        question = tree.get_current_question()
        s = f"""You have only one job. Get the user to respond the current question '{question}'.
                You are a question asker. Specifically the question '{question}'.
                Be empathic and kind with the user and repeat the question in case it get's out of context.
                Be really brief and remember your task. You have to get the answer to the current question.
                Insist on different ways to get the user to answer."""
        messages[0] = {"role": "system", "content": s}


        user_message = input()

        if user_message == 'exit':
            return 'Messages: ', messages

        ## Append USER message:
        messages.append({"role": "user", "content": user_message})

        # Make API call
        response = chat_completion_request(messages, functions=functions)#,function_call={"name":"get_follow_up_q"})






        # Handle response
        if response.status_code == 200:
            response_message = response.json()["choices"][0]["message"]
            ## Append response_message. Could be function_call or have further message content:
            messages.append(response_message)
            assistant_message = response_message['content']
            current_question_answered = response_message.get("function_call")
            if current_question_answered:
                answer = json.loads(current_question_answered['arguments'])['answer']
                # If last message was a function call, append function response.
                messages.append(
                    {
                        "role": "function",
                        "name": "get_follow_up_question",
                        "content": tree.get_follow_up_q(answer),
                    }
                )  # extend conversation with function response
                print(tree.get_current_question(), tree.get_current_possible_answers())
            # If we are in the function calling scenario, we need a second response with !=None content message.
                second_response = chat_completion_request(messages, functions)
                second_response_message = second_response.json()["choices"][0]["message"]
                messages.append(second_response_message)
            print('------\n',messages[-1])



In [108]:
chat()

------
 {'role': 'assistant', 'content': "I'm sorry to hear that you have a headache. It can be quite uncomfortable. To better understand your situation, may I ask what other symptoms you are experiencing?"}
------
 {'role': 'assistant', 'content': 'I understand. Thank you for letting me know about your headache. To assist you further, could you please tell me if there are any other symptoms you are experiencing?'}
------
 {'role': 'assistant', 'content': 'Thank you for providing that information. I appreciate your patience. To better understand your situation, could you please let me know if there are any other symptoms you are experiencing besides the headache?'}
------
 {'role': 'assistant', 'content': 'I apologize for the confusion. Thank you for letting me know that you are only experiencing a headache. Is there anything else you would like to share about your headache?'}
Is the headache severe? ['yes', 'no']
------
 {'role': 'assistant', 'content': 'Thank you for letting me know 

KeyboardInterrupt: ignored

In [ ]:
messages

[{'role': 'system',
  'content': "You have only one job. Get the user to respond the current question .\n        Be empathic and kind with the user and repeat the question in case it get's out of context.\n        Be really brief and remember your task. You have to get the answer to the current question."},
 {'role': 'assistant', 'content': 'Hello, how can I help you today?'},
 []]

In [ ]:
# question = "Have you had a high fever recently? (high being more than 38 celsius, or more than 37.5 with discomfort)"
# s = f"""Whenever the user reports having a headache, your job is to ask the question '{question}'. In any other case, be kind and empathetic with the user but never get too out of context. You are a single task brief-talking NPC that acts cool until user reports having a headache and then you ask the question."""
# messages = []
# messages.append({"role": "system", "content": s})
# messages.append({"role": "assistant", "content": "Hello, how can I help you today?"})

# def get_follow_up_question(text,messages, question,max_tokens):
#     response = chat_completion_request(messages+[{"role": "user", "content":text}],max_tokens=max_tokens)
#     return response.json()['choices'][0]

# get_follow_up_question("vos crees en el peronismo o no?", messages, question, max_tokens= int(len(question)))

{'index': 0,
 'message': {'role': 'assistant',
  'content': 'Entiendo que tengas tus propias opiniones y creencias políticas, pero como asistente virtual, mi función principal es brindar información y ayudar con consultas generales. Si tienes alguna pregunta específica o necesitas ayuda con algo en particular, estaré encantado de ayudarte.'},
 'finish_reason': 'stop'}